# Analyzing the data

In [397]:
# import pandas as pd
# import numpy as np

# # data 5 is bad and 9 is largest

# df = pd.read_csv("odds_datas/test_9_odds_data.csv") # odds_data
# df["T_goals"] = df["H_score"] + df["A_score"]
# df

# STRATEGIES

### All Odds Data

In [422]:
import pandas as pd
import numpy as np

# data 5 is baddest and 9 is largest

df = pd.read_csv("odds_datas/test_16_odds_data.csv") # odds_data

check_list = []
odd_list = []
for _, row in df.iterrows():
    check_list.append((row["week"], len(check_list)+1, row["over_3"], row["H_score"], row["A_score"]))
    if len(check_list) == 10:
        # print(check_list)
        sorted_odds = sorted(check_list, key=lambda x: x[2], reverse=True)
        all_odds = [x[2] for x in sorted_odds]
        for odd in sorted_odds:
            if all_odds.count(odd[2]) == 1:
                odd_list.append(odd)
                break               
        check_list = []

new_df = pd.DataFrame(odd_list, columns=["week", "odd_position_num", "odd", "H_score", "A_score"])
print("Total Stake: ", len(new_df))

print("Mean Odd:", np.mean(new_df["odd"]))
new_df["T_goals"] = new_df["H_score"] + new_df["A_score"]
new_df

Total Stake:  61
Mean Odd: 3.7057377049180324


,week,odd_position_num,odd,H_score,A_score,T_goals
0,15,8,3.43,0,2,2
1,16,3,3.83,3,2,5
2,17,4,3.83,3,1,4
3,18,4,3.76,1,1,2
4,19,4,3.85,3,1,4
...,...,...,...,...,...,...
56,34,10,3.83,2,0,2
57,35,1,3.76,1,0,1
58,36,3,3.76,1,0,1
59,37,1,3.76,0,0,0


In [423]:
count = 1
martingale_list = []
for x in new_df["T_goals"]:
    if x > 3:
        # print(count)
        martingale_list.append(count)
        count = 1             
    else:
        count += 1

print("Total Wins:", len(martingale_list))
print(martingale_list)
print(sorted(martingale_list, reverse=True)[:100])
print("Hightest Martingale Level:", max(martingale_list))

Total Wins: 21
[2, 1, 2, 3, 2, 1, 1, 9, 1, 3, 1, 1, 5, 2, 4, 4, 2, 3, 1, 1, 7]
[9, 7, 5, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]
Hightest Martingale Level: 9


# New over_3 Data

### Martingale vs Single Stake Strategy

In [428]:
import pandas as pd
import numpy as np

# df = new_df.copy()
df = pd.read_csv("auto_trade.csv")
df["T_goals"] = df["H_score"] + df["A_score"] 
print("TOTAL GAMES:", len(df), "\n")

# Split the rows
split_size = 38
array = df.copy().to_numpy()
split_arrays = np.array_split(array, range(split_size, len(array), split_size))

# Convert the split arrays back to DataFrames
split_dfs = [pd.DataFrame(arr, columns=df.columns) for arr in split_arrays]

# Access the split DataFrames
for odd_df in split_dfs:
    # print(split_df)

    bankroll = 100000
    loss_amount = 3700
    highest_loss = -(loss_amount * 3)
    print("Highest Loss :", highest_loss)

    mart_single_target_prtg = 1
    mart_single_target = (mart_single_target_prtg * bankroll) / 100
    print("Single Target -", mart_single_target)

    total_target_prtg = 3
    total_target = (total_target_prtg * bankroll) / 100
    print("Total Target -", total_target, "\n")


    single_bankroll = 0
    single_stake = 1000
    single_bankroll_list = []

    martingale_bankroll = 0
    total_loss = 0
    martingale_bankroll_list = []
    mart_counter = 0
    single_counter = 0

    for _, row in odd_df.iterrows():
        if row["T_goals"] > 3:
            if single_counter <= 5:
                single_bankroll += (row["odd"] - 1) * single_stake
            single_counter = 0
            if mart_counter <= 5:
                martingale_bankroll += mart_single_target 
            mart_counter = 0
        else:
            single_counter += 1
            if single_counter <= 5:
                single_bankroll -= single_stake
            mart_counter += 1
            if mart_counter == 5:
                martingale_bankroll -= loss_amount
        martingale_bankroll_list.append(int(martingale_bankroll))
        single_bankroll_list.append(int(single_bankroll))

    odd_df["Single_Bankroll"] = single_bankroll_list
    odd_df["Check_single"] = np.where(odd_df["Single_Bankroll"] >= total_target, "Achieved", "-")

    odd_df["Martingale_Bankroll"] = martingale_bankroll_list    
    odd_df["Check_Martingale"] = np.where(odd_df["Martingale_Bankroll"] >= total_target, "Achieved", "-")   
    print("TOTAL GAMES:", len(odd_df))


    mart_targets = odd_df[odd_df["Check_Martingale"] == "Achieved"]
    single_targets = odd_df[odd_df["Check_single"] == "Achieved"]

    mart_single_list = [mart_targets, single_targets]

    for temp in range(len(mart_single_list)):
        val = "Martingale_Bankroll" if temp == 0 else "Single_Bankroll"
        if len(mart_single_list[temp]) > 0:
            print("\n--->", val.upper(), "STRATEGY <---")
            print("     Target Amount:", mart_single_list[temp][val].iloc[0])
            print("     Target Position:", (mart_single_list[temp].head(1).index[0]) + 1)

            # Highest values
            print("    ", "-" * 20)
            print("     Highest Profit:", mart_single_list[temp][val].max())
            print("     Highest Profit POSITION:", (odd_df[odd_df[val] == mart_single_list[temp][val].max()].index[0]) + 1)

        if odd_df[val].min() < 0:
            print("    ", "-" * 20)
            print("     Maximum Loss:", odd_df[odd_df[val] < 0][val].min()) 
            print("     Maximum Loss POSITION:", (odd_df[odd_df[val] == odd_df[odd_df[val] < 0][val].min()].index[0]) + 1, "\n")

    print("*" * 100)


TOTAL GAMES: 303 

Highest Loss : -11100
Single Target - 1000.0
Total Target - 3000.0 

TOTAL GAMES: 38

---> MARTINGALE_BANKROLL STRATEGY <---
     Target Amount: 3000
     Target Position: 6
     --------------------
     Highest Profit: 15000
     Highest Profit POSITION: 38

---> SINGLE_BANKROLL STRATEGY <---
     Target Amount: 4610
     Target Position: 3
     --------------------
     Highest Profit: 19220
     Highest Profit POSITION: 34
****************************************************************************************************
Highest Loss : -11100
Single Target - 1000.0
Total Target - 3000.0 

TOTAL GAMES: 38

---> MARTINGALE_BANKROLL STRATEGY <---
     Target Amount: 3300
     Target Position: 22
     --------------------
     Highest Profit: 5300
     Highest Profit POSITION: 26
     --------------------
     Maximum Loss: -1700
     Maximum Loss POSITION: 8 


---> SINGLE_BANKROLL STRATEGY <---
     Target Amount: 4430
     Target Position: 3
     ----------------

In [230]:
# For 3% Target
martingale_strategy = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
single_stake_strategy = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
better_performance = ["S", "S", "S", "M", "S", "M", "S", "S", "S", "S", "S", "S", "M", "", "S", "S"]


In [421]:
odd_df.head(50)

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
0,20.0,8.0,3.85,3.0,1.0,4.0,2850,-,1000,-
1,21.0,5.0,3.76,1.0,0.0,1.0,1850,-,1000,-
2,22.0,6.0,3.76,1.0,0.0,1.0,850,-,1000,-
3,23.0,6.0,3.88,4.0,1.0,5.0,3730,Achieved,2000,-
4,24.0,10.0,3.76,1.0,2.0,3.0,2730,-,2000,-
5,25.0,1.0,3.88,1.0,1.0,2.0,1730,-,2000,-
6,26.0,8.0,3.85,3.0,1.0,4.0,4580,Achieved,3000,Achieved
7,27.0,1.0,3.85,1.0,1.0,2.0,3580,Achieved,3000,Achieved
8,28.0,4.0,3.76,2.0,1.0,3.0,2580,-,3000,Achieved
9,29.0,6.0,3.85,1.0,0.0,1.0,1580,-,3000,Achieved


In [392]:
odd_df[50: 100]

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
50,23.0,2.0,3.83,0.0,2.0,2.0,-520,-,-1400,-
51,24.0,6.0,3.85,2.0,3.0,5.0,-520,-,-1400,-
52,25.0,5.0,3.76,4.0,1.0,5.0,2240,-,-400,-
53,26.0,7.0,3.85,0.0,2.0,2.0,1240,-,-400,-
54,27.0,4.0,3.48,4.0,2.0,6.0,3720,Achieved,600,-
55,28.0,9.0,3.85,2.0,2.0,4.0,6570,Achieved,1600,-
56,29.0,3.0,3.43,2.0,1.0,3.0,5570,Achieved,1600,-
57,30.0,1.0,3.45,1.0,0.0,1.0,4570,Achieved,1600,-
58,31.0,6.0,3.83,0.0,2.0,2.0,3570,Achieved,1600,-
59,32.0,5.0,3.43,1.0,3.0,4.0,6000,Achieved,2600,-


In [322]:
odd_df[100: 150]

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
100,7.0,1.0,3.85,1.0,0.0,1.0,4640,Achieved,-2200,-
101,8.0,1.0,3.40,2.0,1.0,3.0,4640,Achieved,-2200,-
102,9.0,8.0,3.88,1.0,0.0,1.0,4640,Achieved,-2200,-
103,10.0,9.0,3.85,0.0,2.0,2.0,4640,Achieved,-2200,-
104,11.0,8.0,3.85,2.0,1.0,3.0,4640,Achieved,-2200,-
105,12.0,7.0,3.58,0.0,0.0,0.0,4640,Achieved,-2200,-
106,13.0,5.0,3.76,1.0,0.0,1.0,4640,Achieved,-2200,-
107,14.0,10.0,3.88,0.0,0.0,0.0,4640,Achieved,-2200,-
108,15.0,3.0,3.85,0.0,3.0,3.0,4640,Achieved,-2200,-
109,16.0,4.0,3.83,3.0,1.0,4.0,4640,Achieved,-2200,-


In [246]:
odd_df[150: 200]

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
150,37,10,3.40,2,3,5,5600,Achieved,5400,Achieved
151,38,10,3.43,3,2,5,8030,Achieved,6400,Achieved
152,1,10,3.08,0,2,2,7030,Achieved,6400,Achieved
153,2,4,3.85,0,0,0,6030,Achieved,6400,Achieved
154,3,1,3.76,1,5,6,8790,Achieved,7400,Achieved
155,4,5,3.76,3,0,3,7790,Achieved,7400,Achieved
156,5,7,3.29,1,1,2,6790,Achieved,7400,Achieved
157,6,2,3.53,2,0,2,5790,Achieved,7400,Achieved
158,7,6,3.83,2,4,6,8620,Achieved,8400,Achieved
159,8,6,3.85,1,0,1,7620,Achieved,8400,Achieved


In [176]:

count = 1
martingale_list = []
for x in odd_df["T_goals"]:
    if x > 3:
        # print(count)
        martingale_list.append(count)
        count = 1             
    else:
        count += 1

print(martingale_list)
print(sorted(martingale_list, reverse=True)[:50])
print("Hightest Martingale Level:", max(martingale_list))


single_target = 1000
loss_amount = 3700

win_length = sorted(martingale_list, reverse=True)
total_diff = 0
for x in win_length:
    if x > 5:
        total_diff += 1
    else:
        break
print("\nTotal Played:", len(martingale_list))
print("Total Lost:", total_diff)
won = len(martingale_list) - total_diff
print("Total Won:", won)
print("Profit/Loss Amount:", (won * single_target) - (total_diff * loss_amount))

[1, 4, 2, 4, 10, 1, 11, 2, 7, 3, 2, 6, 8, 6, 1, 8, 1, 3, 3, 1, 2, 2, 2, 15, 1, 3, 3, 3, 6, 1, 4, 1, 1, 1, 3, 4, 3, 2, 1, 2, 3, 4, 6, 1, 2, 1, 4, 9, 2, 2, 2, 1, 1, 2, 3, 9, 31, 4, 1, 4, 3, 3, 9, 4, 14, 4, 1, 1, 1, 2, 6, 3]
[31, 15, 14, 11, 10, 9, 9, 9, 8, 8, 7, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Hightest Martingale Level: 31

Total Played: 72
Total Lost: 16
Total Won: 56
Profit/Loss Amount: -3200


## Single Stake Strategy

In [154]:
targets = odd_df[odd_df["Check_single"] == "Achieved"]
print("Total Games:", len(odd_df))
print("Target Position:", (targets.head(1).index[0]) + 1)

# Highest Profit
print("Highest Profit:", targets["Single_Bankroll"].max())
print("Highest Profit POSITION:", (odd_df[odd_df["Single_Bankroll"] == targets["Single_Bankroll"].max()].index[0]) + 1)

Total Games: 27
Target Position: 26
Highest Profit: 6820
Highest Profit POSITION: 26


In [155]:
targets

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
25,8,8,3.85,4,2,6,6820,Achieved,-18500,-
26,9,9,3.42,0,1,1,5820,Achieved,-18500,-
